In [3]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


drive = webdriver.Chrome()
def get_data_bikubben(url):
    
    """This function will collect the grant information provided from the Bikubben foundation grant database with the url: https://www.bikubenfonden.dk/bevillingsdatabase
    The XPATHS are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction.
    
    The information is saved in the lists of the following groups:
    year_l = []
    project_category_l = []
    receiver_name_title = []
    description_l = []
    amount_L = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://www.bikubenfonden.dk/bevillingsdatabase)
    return: Dataframe with text extracted
    
    
    """
    #inititating lists
    year_l = [] 
    project_category_l = []
    receiver_name_title = []
    description_l = []
    amount_L = []
    
    
    drive.get(url)
    drive.maximize_window() #maximizing window
    time.sleep(5)
    
    action = ActionChains(drive) #action initiator 

    
    try:
        for i in range(1,1165): 
            if i % 30 == 0:
                ActionChains(drive).move_to_element(drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[1165]/button")).perform()  #moving to the first line to find the button to click
                elements_first = WebDriverWait(drive, 10).until(EC.element_to_be_clickable((By.XPATH,f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[1165]/button")))  
                elements_first.click()
                time.sleep(5)
                
            try:  #Trying to add the information to the right lists
                action.move_to_element(drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i+1}]/div/div[1]/h3")).perform()  #moving to the line of interest
                year = drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[2]/div[2]").text  #yearlist
                project_cat = drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[2]/div[1]").text  #project category list
                rec_name = drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[1]/h3").text  #name and title
                amount = drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[1]/div").text #receiver and title
                ActionChains(drive).move_to_element(drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i+1}]/div/div[1]")).perform()
                year_l.append(year) #appending to the lists
                project_category_l.append(project_cat)
                receiver_name_title.append(rec_name)
                amount_L.append(amount)
                
                
                try:
                    element_second = WebDriverWait(drive, 12).until(EC.element_to_be_clickable((By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[1]")))  #wait for line to be clickable
                    element_second.click()
                    description = drive.find_element(By.XPATH, f"/html/body/div[2]/div/main/div[1]/article/div/div[2]/div/div[2]/div/div[2]/div/div[2]/div[2]/div[{i}]/div/div[3]/div").text
                    description_l.append(description)
                except ElementClickInterceptedException:  #exception when the error can't be clicked
                    description = "NA"
                    description_l.append(description)
                    print(i)
                    continue
                    

                
            except NoSuchElementException:
                year = "NA"
                project_cat = "NA"
                rec_name = "NA"
                description = "NA"
                amount = "NA"
                year_l.append(year)#appending to lists
                project_category_l.append(project_cat)
                receiver_name_title.append(rec_name)
                description_l.append(description)
                amount_L.append(amount)
            
    except NoSuchElementException: #the error is cathed to avoid programme crashing
        print("ended loop")


    df_final = pd.DataFrame(list(zip(year_l, receiver_name_title, amount_L, description_l, project_category_l)), columns=["Year",'Reciever and Title','Grant size (DKK)', 'Description', "Project Category"])

    return df_final

biglist = get_data_bikubben("https://www.bikubenfonden.dk/bevillingsdatabase")
pd.set_option('display.max_columns', None)

biglist.to_csv('bikubben_data.csv', index=True)  #The dataframe is saved to a csv file

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))